In [ ]:
# STEP 1: Install the Google Generative AI SDK
!pip install -q google-generativeai

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "ENTER API KEY"


In [ ]:
# text generation from the model.
from google import genai

client = genai.Client()
#response = client.models.generate_content( model="gemini-2.5-pro",contents="Hey")
response = client.models.generate_content( model="gemini-2.5-flash",contents="What iS CURRENT date AND TIME IN DELHI")
print(response.text)

Today in Delhi, India, it is:

**[Current Day of the Week], [Current Month] [Current Day], [Current Year]**
**[Current Time in HH:MM:SS AM/PM] IST**

*(For example: Tuesday, May 7, 2024 at 10:30:00 AM IST)*

Indian Standard Time (IST) is UTC+5:30.


In [ ]:
response = client.models.generate_content( model="gemini-2.5-flash",contents="What do you know about Operation Sindoor By Indian Army.")
print(response.text)

I cannot find any widely known or officially documented Indian Army operation named "Operation Sindoor."

It's possible that:
*   The name is incorrect or a misunderstanding.
*   It refers to a very localized, unofficial, or fictional event (e.g., in a book, movie, or story).
*   It's an internal code name that was never publicly disclosed.

Military operations usually have specific codenames, and "Sindoor" (vermilion, a traditional Hindu symbol of marriage) would be a highly unusual choice for an official military operation name.

If you have more context about where you heard this name, it might help to clarify what it refers to.


In [ ]:
response_1 = client.models.generate_content( model="gemini-2.5-flash",contents="What is current temperature of Chandigarh")
print(response_1.text)

As of my last update, the current temperature in Chandigarh is approximately **28°C (82°F)**, with conditions described as **hazy**.

For the most real-time and up-to-the-minute information, I recommend checking a live weather service or app.


In [ ]:
response_2 = client.models.generate_content( model="gemini-2.5-flash",contents="Send an email :- 'Sorry' to the mail address:-'sati76857@gmail.com'")
print(response_2.text)


I cannot directly send emails. However, I can provide you with the content to send it yourself.

**To:** sati76857@gmail.com
**Subject:** Sorry

**Body:**
Sorry.


# The Core Problem: LLMs are Isolated Brains

By default, a large language model (LLMs) is like a brilliant brain in a jar. It possesses vast amounts of knowledge, but it's static and disconnected from the real world.

One major limitation is its knowledge cutoff, its information is frozen at the point when its training data was collected. That means it doesn't know the current date, today's weather, or recent news.

Another key limitation is its inability to act. The model cannot perform actions such as sending an email, booking a flight, checking the temperature of your city, or querying a private database. It can only generate text.

In [ ]:
# As we known that gemini model is very strong .that gives text , image and audio even solves the logical questions .
# But its builds for text generation , image generation ,
# as you can see we asked about time and operation sindoor and he could not answer it  because of its to upto date
# LLM have boundaries that's why unable to think outside of box


# so we use Function Calling

#FUNCTION CALLING

Function calling is the mechanism that gives this brain "hands and senses" to interact with the outside
world external systems and APIs in a structured way.

It is an feature that allows developers to create an function for an specific task , pass it to the model and model based
on the problem asked use that function and give the output.

In [ ]:
import datetime
import pytz # time-zone

tz= pytz.timezone("Asia/Kolkata")  ##Indian timezone
print(tz)

now = datetime.datetime.now(tz)  # non-readable form

Asia/Kolkata


In [ ]:
print(now.strftime("%Y-%m-%d"))          # Date
print(now.strftime("%I:%M:%S %p"))       # Time (HRS,MINT,SEC,AM/PM)

2025-10-21
09:43:02 AM


In [ ]:
# TYPE HINTING ( which type of input you give and which type of output you want)
# Description
# these both are not compulsoiry but recommended
# Benifit of using this
# when you call your model and pass the function ,model easily understand that which model to use to solve this problem

In [ ]:
# type-hinting
# description

def get_current_datetime(timezone: str ="UTC") -> dict:
  """Fetches  the current date and time  for the given  timezone.

  Args:
     timezone(str): the timezone  name (eg="Asia/Kolkata","UTC")

  Returns:
     dict: A dictionary containing  the current date and time  in the specified timezone,
          or an error message  if the timezone is invalid.
  """

  print(f"-- Function  get_current_datetime is called for the timezone {timezone}")  # when we call function this line will print
  try:
     tz= pytz.timezone(timezone)
     now = datetime.datetime.now(tz)

     return{
         "date":now.strftime("%Y-%m-%d"),
         "time":now.strftime("%I:%M:%S %p")
     }
  except pytz.UnknownTimeZoneError:
     return {"error: Invalid Timezone Specified."}

In [ ]:
get_current_datetime("Asia/Kolkata")

-- Function  get_current_datetime is called for the timezone Asia/Kolkata


{'date': '2025-10-21', 'time': '09:43:02 AM'}

In [ ]:
from google.genai import types
client= genai.Client()

response= client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the current date and time in china",
    config= types.GenerateContentConfig(
        tools=[get_current_datetime]
    )
)

print(response.text)

-- Function  get_current_datetime is called for the timezone Asia/Shanghai
The current date and time in China is 2025-10-21 12:13:03 PM.


In [ ]:
# understand whats going on backgroud , so add thinking_config class
from google.genai import types
client= genai.Client()

response_3= client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is current date and time in Delhi",  # if you ask any other question function will not run
    config= types.GenerateContentConfig(
        tools=[get_current_datetime],
        thinking_config= types.ThinkingConfig(
            include_thoughts=True
        )
    )
)



-- Function  get_current_datetime is called for the timezone Asia/Kolkata


In [ ]:
response_3

GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text='What is current date and time in Delhi'
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          text="""**Determining the Current Time in Delhi**

Okay, so I need to find out the current date and time in Delhi.  It's clear the user is asking for this specifically, and I know I have a `get_current_datetime` tool that can handle timezones.  My knowledge base tells me Delhi is in the "Asia/Kolkata" timezone, so the correct approach is to call that tool with the `timezone` parameter set to "Asia/Kolkata".  That should give me the answer I need.
""",
          thought=True
        ),
        Part(
          function_call=FunctionCall(
            args={<... 1 item at Max depth ...>},
            name='get_current_datetime'
          ),
          thought_signature=b'\n\xb2\x02\x01\xd1\xed\x8aoG\x8a\xf9\x01v\xe40h\xf8\x9b\x96\xde~

In [ ]:
response_3.automatic_function_calling_history[0]   # role = 'user' its you

UserContent(
  parts=[
    Part(
      text='What is current date and time in Delhi'
    ),
  ],
  role='user'
)

In [ ]:
response_3.automatic_function_calling_history[1]

Content(
  parts=[
    Part(
      text="""**Determining the Current Time in Delhi**

Okay, so I need to find out the current date and time in Delhi.  It's clear the user is asking for this specifically, and I know I have a `get_current_datetime` tool that can handle timezones.  My knowledge base tells me Delhi is in the "Asia/Kolkata" timezone, so the correct approach is to call that tool with the `timezone` parameter set to "Asia/Kolkata".  That should give me the answer I need.
""",
      thought=True
    ),
    Part(
      function_call=FunctionCall(
        args={
          'timezone': 'Asia/Kolkata'
        },
        name='get_current_datetime'
      ),
      thought_signature=b'\n\xb2\x02\x01\xd1\xed\x8aoG\x8a\xf9\x01v\xe40h\xf8\x9b\x96\xde~J@\x83T\x01\x07\x0c\x8f!\xefWds\x932E\xa2\xc9\xd6q\xa8\xfd\x86I\x17T?X\xbf\x18\x18u\xadngoqu\xe9\xe3\xab\xc8$\x17\x04/\x8ds\x00\xdc\xa1\x87\xc9I\xca\xc2\xcb5\xfc\x8c\x92\x19y\xad\xc6\xa7\x08A\xa5x@\xc3\x92\xcd\xc1X...'
    ),
  ],
  role='mo

In [ ]:
response_3.automatic_function_calling_history[1].parts[0]

Part(
  text="""**Determining the Current Time in Delhi**

Okay, so I need to find out the current date and time in Delhi.  It's clear the user is asking for this specifically, and I know I have a `get_current_datetime` tool that can handle timezones.  My knowledge base tells me Delhi is in the "Asia/Kolkata" timezone, so the correct approach is to call that tool with the `timezone` parameter set to "Asia/Kolkata".  That should give me the answer I need.
""",
  thought=True
)

In [ ]:
response_3.automatic_function_calling_history[1].parts[1]  #  resend to user/system   makes a JSON structure

# gemini -- function execute(X)
# internal thinking :- function, arguments [value]
# after making sure about internal thinking now , it will create a JSON structure

# gemini- json schema :- function , arguments , value  ----->system.
# gemini have no power to execute any function


# system -- command  -- [gemini : function]  , system

Part(
  function_call=FunctionCall(
    args={
      'timezone': 'Asia/Kolkata'
    },
    name='get_current_datetime'
  ),
  thought_signature=b'\n\xb2\x02\x01\xd1\xed\x8aoG\x8a\xf9\x01v\xe40h\xf8\x9b\x96\xde~J@\x83T\x01\x07\x0c\x8f!\xefWds\x932E\xa2\xc9\xd6q\xa8\xfd\x86I\x17T?X\xbf\x18\x18u\xadngoqu\xe9\xe3\xab\xc8$\x17\x04/\x8ds\x00\xdc\xa1\x87\xc9I\xca\xc2\xcb5\xfc\x8c\x92\x19y\xad\xc6\xa7\x08A\xa5x@\xc3\x92\xcd\xc1X...'
)

In [ ]:
response_3.automatic_function_calling_history[2] # executed by user



Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        name='get_current_datetime',
        response={
          'result': {
            'date': '2025-10-21',
            'time': '09:43:06 AM'
          }
        }
      )
    ),
  ],
  role='user'
)

In [ ]:
response_3.candidates

# after excecuting
# output of this function ---> goes as input to the model
# to convert into a structured form

[Candidate(
   content=Content(
     parts=[
       Part(
         text='The current date and time in Delhi is 2025-10-21 09:43:06 AM.'
       ),
     ],
     role='model'
   ),
   finish_reason=<FinishReason.STOP: 'STOP'>,
   index=0
 )]

In [ ]:
response_3.candidates

[Candidate(
   content=Content(
     parts=[
       Part(
         text='The current date and time in Delhi is 2025-10-21 09:43:06 AM.'
       ),
     ],
     role='model'
   ),
   finish_reason=<FinishReason.STOP: 'STOP'>,
   index=0
 )]

# Qucik revision


1. make a function
2. LLM ko intialize kartey time function kiya
3. LLM ko ek prompt dia
4. it will now decide , can i give output or not
5. if yes then it will give output
6. if no, then it will decide that which function to use
7. after deciding function
   it study's the function , think arrgument of the function
after deciding which model and function it will use
8. now, it will create JSON schema
JSON schema ---> argument ---> argument values
then this jJSON schema pass to the system
9. after to the system function will call
gives output ---> fetched by model ( not in the form of human readable)
10. model ---> process the outputgenerate into the textual generation

In [ ]:
# PARALLEL FUNCTION CALLING.
# LLM -- AUTOMATICALLY DECIDE  WHICH FUNCTION TO CALL.
# MULTIPLE : LLM CAN DO.

# USER :- TELL ME CURRENT DATE TIME AND  TEMPERATURE  OF DELHI.

In [ ]:
def get_temperature(city: str) -> dict:
    """
    function that returns temperature data for a given city.

    Args:
        city (str): Name of the city

    Returns:
        dict: A dictionary with temperature and condition
    """
    print(f"--- Function get_temperature ({city} Called")  # when function call it will print
    # Hardcoded fake temperature data
    fake_data = {
        "New York": {"temperature": "30°C", "condition": "Sunny"},
        "Delhi": {"temperature": "36°C", "condition": "Hot and Dry"},
        "London": {"temperature": "22°C", "condition": "Cloudy"},
        "Tokyo": {"temperature": "28°C", "condition": "Humid"},
        "Paris": {"temperature": "25°C", "condition": "Breezy"},
    }

    # Return fake data if city exists, else give default
    return fake_data.get(city, {"temperature": "Unknown", "condition": "Data not available"})

In [ ]:
from google.genai import types
client= genai.Client()

response_4= client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the current date and time and temperature for Delhi",
    config= types.GenerateContentConfig(
        tools=[get_current_datetime,get_temperature],
        thinking_config= types.ThinkingConfig(
            include_thoughts=True
        )
    )
)


-- Function  get_current_datetime is called for the timezone Asia/Kolkata
--- Function get_temperature (Delhi Called


In [ ]:
print(response_4.text)

The current date and time for Delhi is 2025-10-21, 09:47:23 AM and the temperature is 36°C, Hot and Dry.


In [ ]:
from google.genai import types
client= genai.Client()

response_4= client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What is the current date and time and temperature for new york",
    config= types.GenerateContentConfig(
        tools=[get_current_datetime,get_temperature],
        thinking_config= types.ThinkingConfig(
            include_thoughts=True
        )
    )
)


-- Function  get_current_datetime is called for the timezone America/New_York
--- Function get_temperature (New York Called


In [ ]:
response_4.text

'The current date and time for New York is 2025-10-21 12:26:42 AM and the temperature is 30°C and it is Sunny.'

In [ ]:
# backend working

response_4

GenerateContentResponse(
  automatic_function_calling_history=[
    UserContent(
      parts=[
        Part(
          text='What is the current date and time and temperature for new york'
        ),
      ],
      role='user'
    ),
    Content(
      parts=[
        Part(
          text="""**Gathering Information for the User's Query**

Okay, so I need to get the user what they're looking for. They want the date, time, and temperature in New York.  Seems straightforward. I can leverage my tools: `get_current_datetime` for the date and time, and `get_temperature` to fetch the temperature. 

For `get_current_datetime`, I'll need to specify the timezone, and "America/New_York" is the clear choice for New York.  No surprises there. And for `get_temperature`, I just need the city, which is "New York" – easy peasy.

I'll call `get_current_datetime` first, followed by `get_temperature`, and then I'll put the results together in a concise format to give the user exactly what they requested. 